How to find Tuesdays late in the month?

In [4]:
function getTuesdays(month, year) {
    var d = new Date(year, month, 1),
        tuesdays = [];

    d.setDate(d.getDate() + (9 - d.getDay()) % 7)
    while (d.getMonth() === month) {
        tuesdays.push(new Date(d.getTime()));
        d.setDate(d.getDate() + 7);
    }

    return tuesdays;
}

var month = (new Date()).getMonth();
var results = [];
for(var m = month; month <= 12; month++) {
    results = results.concat(getTuesdays(month, 2017).filter((d) => d.getDate() > 25));
}
console.log(results.map(d => d.getDate() + '/' + (d.getMonth()+1)).join('\r\n'))

How to authenticate to Google APIs?



In [1]:
var execSync = require('child_process').execSync;
try {
    require.resolve('googleapis');
    require.resolve('google-auth-library');
} catch (e) {
    execSync('npm install googleapis google-auth-library');
}


// Load client secrets from a local file.
var path = require('path');
var fs = require('fs');
var google = require('googleapis');
var googleAuth = require('google-auth-library');

// If modifying these scopes, delete your previously saved credentials
// at ~/.credentials/calendar-nodejs-quickstart.json
var SCOPES = ['https://www.googleapis.com/auth/calendar.readonly'];
var TOKEN_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.credentials');
var TOKEN_PATH = path.join(TOKEN_DIR, 'calendar-nodejs-quickstart.json');
var SECRET_PATH = path.join(TOKEN_DIR, 'client_secret.json');
var credentials = JSON.parse(fs.readFileSync(SECRET_PATH));

/**
 * Store token to disk be used in later program executions.
 *
 * @param {Object} token The token to store to disk.
 */
function storeToken(token) {
  try {
    fs.mkdirSync(TOKEN_DIR);
  } catch (err) {
    if (err.code != 'EEXIST') {
      throw err;
    }
  }
  fs.writeFile(TOKEN_PATH, JSON.stringify(token));
  console.log('Token stored to ' + TOKEN_PATH);
}

/**
 * Create an OAuth2 client with the given credentials, and then execute the
 * given callback function.
 *
 * @param {Object} credentials The authorization client credentials.
 * @param {function} callback The callback to call with the authorized client.
 */
var authorize = () => {
    var clientSecret = credentials.installed.client_secret;
    var clientId = credentials.installed.client_id;
    var redirectUrl = credentials.installed.redirect_uris[0];
    var auth = new googleAuth();
    var oauth2Client = new auth.OAuth2(clientId, clientSecret, redirectUrl);

    try {
        // Check if we have previously stored a token.
        var token = fs.readFileSync(TOKEN_PATH);
        oauth2Client.credentials = JSON.parse(token);
        return Promise.resolve(oauth2Client);
    } catch (e) {
        if(e.code !== 'ENOENT') {
            throw e;
        } else {
            var authUrl = oauth2Client.generateAuthUrl({
                    access_type: 'offline',
                    scope: SCOPES
                });
            return authorizeSelenium(authUrl)
                .then(token => {
                    oauth2Client.credentials = token;
                    storeToken(token);
                    return oauth2Client;
                });
        }
    }
};
(authorize);


[Function: authorize]

In [2]:
var path = require('path');
var corePath = path.resolve(path.join(__dirname, '..', 'Core'));
var importer = require(corePath);
var interpretAllPath = path.join('..', 'Core', 'How to interpret all notebooks.ipynb');
var authorizeSelenium = (authUrl) => {
    var seleniumDocker = null,
        getSeleniumServer = null,
        createWebdriverClient = null,
        client = null, 
        results = null, newToken = null;
    console.log('Authorizing: ' + authUrl);
    return importer.importNotebook(interpretAllPath)
        .then((f) => Promise.all([
            interpret('selenium docker'),
            interpret('set up selenium server'),
            interpret('webdriver client node'),
            interpret('decrypt password'),
            interpret('log in Google'),
        ]))
        .then(r => r.map(r => r[0]))
        .then(r => interpretObject(r))
        .then(r => {
            results = r;
            seleniumDocker = eval(r[0].code);
            getSeleniumServer = eval(r[1].code);
            console.log('Restarting docker');
            return getSeleniumServer()
        })
        .then(() => {
            createWebdriverClient = eval(results[2].code);
            client = createWebdriverClient('localhost', 4444);
            eval(results[3].code);
            eval(results[4].code);
            return client
                .url(authUrl)
                .loginGoogle()
                .waitForVisible('#submit_approve_access content', 3000)
                .click('#submit_approve_access content')
                .waitForVisible('#code', 3000)
                .getValue('#code')
        })
        .then(code => {
            newToken = code;
            console.log(newToken);
            return oauth2Client.getToken(newToken);
        });
};
(authorizeSelenium);

[Function: authorizeSelenium]

In [3]:
// Authorize a client with the loaded credentials, then call the
  // Google Calendar API.
$$.async();
authorize()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



Authorizing: https://accounts.google.com/o/oauth2/auth?access_type=offline&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&response_type=code&client_id=716181333443-4du7obudqidc0dned9j4sckupp4b76ju.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob
Restarting docker
Already imported
[ 'How to import a notebook.ipynb',
  'How to interpret all notebooks.ipynb' ]
Initializing webdriver on localhost
4/w0XEG63tG7F1a02RY9OD6-pqT_5m6I-wiKY5Ub7YtuU


ReferenceError: oauth2Client is not defined

How often does a google calendar event occur?

In [1]:

/**
 * Lists the next 10 events on the user's primary calendar.
 *
 * @param {google.auth.OAuth2} auth An authorized OAuth2 client.
 */
var resultEvents = [];
function listEvents(auth) {
  var calendar = google.calendar('v3');
  calendar.events.list({
    auth: auth,
    //calendarId: 'p4685f6at8ccctenocjlqgnqss@group.calendar.google.com',  
    //calendarId: 'rqqhq3me7ggg47chafhmeo6m3s@group.calendar.google.com',
      calendarId: 'primary',
      q: 'work on SOS',
      timeMin: '2017-05-20T10:00:00-07:00',
    maxResults: 100,
    singleEvents: true,
    orderBy: 'startTime'
  }, function(err, response) {
    if (err) {
      console.log('The API returned an error: ' + err);
      return;
    }
    var events = response.items;
    if (events.length == 0) {
      console.log('No upcoming events found.');
    } else {
        resultEvents = []; // reset the array
      for (var i = 0; i < events.length; i++) {
        var event = events[i];
        var start = event.start.dateTime || event.start.date;
          resultEvents[resultEvents.length] = {start: new Date(start), event: event};
        console.log('%s - %s', start, event.summary);
      }
    }
  });
}


In [21]:
// sum up events
var total = 0;
resultEvents.forEach(e => {
    if(typeof e.event.end === 'undefined') {
        console.log(e.event);
    }
    total += new Date(e.event.end.dateTime).getTime() - new Date(e.event.start.dateTime).getTime();
});

console.log(total / 1000 / 60 / 60);

In [25]:
var execSync = require('child_process').execSync;
try {
    require.resolve('d3-node');
} catch (e) {
    execSync('npm install d3-node');
}

// subuffertract the events from the previous one, if it is greater than 1 months it can't help our graph
var contributing = [];
resultEvents.sort((a, b) => a.start - b.start);
resultEvents.forEach((e, i) => {
    if(i == 0) {
        return;
    }
    var diff = (e.start.getTime() - resultEvents[i - 1].start.getTime()) / 1000 / 60 / 60 / 24;
    //if(diff < 30 && diff > 0) {
        contributing[contributing.length] = {days: diff, start: e.start, event: e.event, i: contributing.length};
    //}
});

var D3Node = require('d3-node');
var d3n = new D3Node(); // initializes D3 with container element 
var d3 = d3n.d3;
console.log(contributing.map(e => e.days));

// set the dimensions and margins of the graph
var margin = {top: 20, right: 20, bottom: 30, left: 50},
    width = 960 - margin.left - margin.right,
    height = 500 - margin.top - margin.bottom;

// set the ranges
var x = d3.scaleTime().range([0, width]);
var y = d3.scaleLinear().range([height, 0]);

// define the line
var valueline = d3.line()
    .x(function(d) { return x(d.start); })
    .y(function(d) { return y(d.days); });

// append the svg obgect to the body of the page
// appends a 'group' element to 'svg'
// moves the 'group' element to the top left margin
var svg = d3n.createSVG(width + margin.left + margin.right,
                        height + margin.top + margin.bottom)
  .append("g")
    .attr("transform",
          "translate(" + margin.left + "," + margin.top + ")");

// Get the data

// Scale the range of the data
x.domain(d3.extent(contributing, function(d) { return d.start; }));
y.domain([0, d3.max(contributing, function(d) { return d.days; })]);

// Add the valueline path.
svg.append("path")
  .data([contributing])
  .attr("class", "line")
    .attr("fill", "none")
    .attr("stroke", "#000")
  .attr("d", valueline);

// Add the X Axis
svg.append("g")
  .attr("transform", "translate(0," + height + ")")
  .call(d3.axisBottom(x));

// Add the Y Axis
svg.append("g")
  .call(d3.axisLeft(y));

$$.svg(d3n.svgString());